In [1]:
from typing import List, Literal, Annotated
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# --- 1. Robust Schemas ---
class ResearchReport(BaseModel):
    title: str = Field(description="Catchy title")
    summary: str = Field(description="2-sentence summary")
    sources: List[str] = Field(description="List of URLs")
    confidence_score: float = Field(description="0 to 1")

class ValidationResult(BaseModel):
    is_valid: bool = Field(description="True if facts are dated for 2026")
    critique: str = Field(description="Explanation of gaps")
    # Using a literal ensures the LLM picks ONLY one of these two strings
    needed_action: Literal["finalize", "research_more"] = Field(
        description="Must be 'finalize' or 'research_more'"
    )

# --- 2. Setup ---
llm = ChatGroq(model="qwen-2.5-32b", temperature=0) # Use 0 temp for extraction
tools = [TavilySearchResults(max_results=3)]
memory = InMemorySaver()

# Initialize Middleware
sum_middleware = SummarizationMiddleware(
    model=llm,
    trigger=("messages", 5), # Summarize after 5 messages
    keep=("messages", 2)      # Keep the most recent 2
)

# --- 3. The Agents ---
# The researcher agent uses the tools to browse
research_agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,
    middleware=[sum_middleware],
    system_prompt="You are a 2026 research specialist. Search and find hard facts."
)

# Structured Parsers
research_parser = llm.with_structured_output(ResearchReport)
validation_parser = llm.with_structured_output(ValidationResult)

# --- 4. Orchestration Logic ---
def run_validated_research(query: str):
    config = {"configurable": {"thread_id": "starship_2026"}}
    
    # Step 1: Research
    print("🚀 Researching...")
    state = research_agent.invoke({"messages": [("user", query)]}, config=config)
    raw_draft = state["messages"][-1].content
    
    # Step 2: Validate (Using the parser to enforce the schema)
    print("🔍 Validating...")
    # We add a clear instruction to follow the schema strictly
    v_prompt = f"Critique this research for 2026 accuracy. Output must include needed_action.\n\n{raw_draft}"
    
    try:
        validation = validation_parser.invoke(v_prompt)
    except Exception:
        # Emergency fallback if Groq still throws a 400
        validation = ValidationResult(is_valid=False, critique="Schema failed", needed_action="research_more")

    if not validation.is_valid:
        print(f"⚠️ Validation Failed: {validation.critique}")
        # Step 3: Revise
        state = research_agent.invoke(
            {"messages": [("user", f"Fix your report based on this: {validation.critique}")]},
            config=config
        )
        raw_draft = state["messages"][-1].content

    # Step 4: Final Structure
    return research_parser.invoke(raw_draft)

# Execution
final_report = run_validated_research("SpaceX Starship updates for late 2026")
print(f"\n--- {final_report.title} ---\n{final_report.summary}")


C:\Users\irams\AppData\Local\Temp\ipykernel_14892\753023525.py:26: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tools = [TavilySearchResults(max_results=3)]


🚀 Researching...


BadRequestError: Error code: 400 - {'error': {'message': 'The model `qwen-2.5-32b` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}